In [2]:
#  ! pip install --upgrade langchain langchain-community langchainhub langchain-openai openinference-instrumentation-langchain langchain-text-splitters faiss-cpu 'arize-phoenix[evals]' 


In [3]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [8]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [10]:
import phoenix as px
session = px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [11]:
# ORIGINAL CODE:
# from phoenix.trace.langchain import LangChainInstrumentor

# UDPATED CODE:

from openinference.instrumentation.langchain import LangChainInstrumentor

LangChainInstrumentor().instrument()

Attempting to instrument while already instrumented


In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)

# added to handle unicode error
# text_loader_kwargs={'autodetect_encoding': True}
text_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader("../sports_fiascos", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
docs = loader.load_and_split(text_splitter=text_splitter)

FileNotFoundError: Directory not found: '../sports_fiascos'

In [ ]:
# import faiss
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(
    docs, 
    embeddings)

In [ ]:
retriever = db.as_retriever()

In [ ]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "search_sports_fiascos",
    "Searches and returns excerpts from Wikipedia entries of sports fiascos.",
)
tools = [tool]

In [ ]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

/workspaces/ai_agents_cookbooks/.venv/lib/python3.12/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, verbose=True)

In [ ]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [ ]:
result = agent_executor.invoke(
    {
        "input": "Which sporting fiasco resulted in the most fiscal damage?"
    }
)

In [ ]:
result["output"]

"The search results provided information about a sporting fiasco involving the 2023 Rugby World Cup and the 2024 Summer Olympics in France. The incident led to significant fiscal damage and harm to the nation's reputation. The UEFA inquiry highlighted organizational and safety failures, with both UEFA and French authorities being criticized for attempting to avoid responsibility for the chaos that ensued. The impact of the fiasco extended to later tours, with reports of fans being assaulted, harassed, and robbed during the events.\n\nWhile the exact fiscal damage was not quantified in the search results, the repercussions of this sporting fiasco were substantial in terms of financial implications and reputation damage."